In [1]:
%load_ext autoreload
%autoreload 2
import my_baselines

In [2]:
with open('train.json') as f:
    data = f.readlines()

import ast
data = [ast.literal_eval(x) for x in data]

In [3]:
# data[0]

In [13]:
import numpy as np

class MF():

    def __init__(self, R, K, alpha, beta, iterations, all_samples=None):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        
        # Create a list of training samples
        if all_samples is None:
            self.all_samples = [
                (i, j, self.R[i, j])
                for i in range(self.num_users)
                for j in range(self.num_items)
                if self.R[i, j] > 0
            ]
        else:
            self.all_samples = all_samples

    def train(self):
        import time
        start_time = time.time()
        
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        
        

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.all_samples)
            self.samples = self.all_samples[:128]
            self.sgd()
            
            if (i+1) % 20 == 0:
                mse = self.mse()
                training_process.append((i, mse))
                cur_time = time.time() - start_time
                print("%dm:%ds Iteration: %d ; error = %.4f" % (int(cur_time/60), int(cur_time%60) ,i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [5]:
# R = np.array([
#     [5, 3, 0, 1],
#     [4, 0, 0, 1],
#     [1, 1, 0, 5],
#     [1, 0, 0, 4],
#     [0, 1, 5, 4],
# ])

# mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)
# mf.train()
# mf.full_matrix()

In [6]:
class Mat:
    def __init__(self):
        self.user2index = {}
        self.item2index = {}
        self.n_user = 0
        self.n_item = 0
        
        self.user2item = {}
        self.item2user = {}
        self.all_rating = 0.
        self.all_cnt = 0
        
        self.all_samples = []
        
    def addEntry(self, entry):
        u = entry['reviewerID']
        i = entry['itemID']
        r = entry['rating']
        
        self.all_rating += r
        self.all_cnt += 1
        
        if u not in self.user2index:
            self.user2index[u] = self.n_user
            self.n_user += 1
        if i not in self.item2index:
            self.item2index[i] = self.n_item
            self.n_item += 1
            
        self.all_samples.append((self.user2index[u], self.item2index[i], r))
        
        if u not in self.user2item:
            self.user2item[u] = [(i,r)]
        else:
            self.user2item[u].append((i,r))
            
        if i not in self.item2user:
            self.item2user[i] = [(u,r)]
        else:
            self.item2user[i].append((u,r))
    
    def addEntries(self, entries):
        for e in entries:
            self.addEntry(e)
        self.all_avg = self.all_rating / self.all_cnt
    
    def ratingMat(self):
        self.R = np.zeros([self.n_user, self.n_item])
        for u, value in self.user2item.items():
            for v in value:
                i, r = v
                user_index = self.user2index[u]
                item_index = self.item2index[i]
                self.R[user_index, item_index] = r
                

In [7]:
RA = Mat()
RA.addEntries(data)
RA.ratingMat()

In [8]:
mfa = MF(RA.R, K=500, alpha=0.1, beta=0.01, iterations=2000, all_samples=RA.all_samples)

In [9]:
mfa.train()

Iteration: 5 ; error = 494.6643
Iteration: 10 ; error = 494.3951
Iteration: 15 ; error = 493.8775
Iteration: 20 ; error = 493.3205
Iteration: 25 ; error = 492.7628
Iteration: 30 ; error = 492.3618
Iteration: 35 ; error = 491.8166
Iteration: 40 ; error = 491.2916
Iteration: 45 ; error = 490.8752
Iteration: 50 ; error = 490.4412
Iteration: 55 ; error = 490.0333
Iteration: 60 ; error = 489.6977
Iteration: 65 ; error = 489.2581
Iteration: 70 ; error = 488.8295
Iteration: 75 ; error = 488.4153
Iteration: 80 ; error = 487.9820
Iteration: 85 ; error = 487.6733
Iteration: 90 ; error = 487.2769
Iteration: 95 ; error = 486.8200
Iteration: 100 ; error = 486.3582
Iteration: 105 ; error = 485.8987
Iteration: 110 ; error = 485.5534
Iteration: 115 ; error = 485.2397
Iteration: 120 ; error = 484.8973
Iteration: 125 ; error = 484.5847
Iteration: 130 ; error = 484.3004
Iteration: 135 ; error = 483.9524
Iteration: 140 ; error = 483.5147
Iteration: 145 ; error = 483.2591
Iteration: 150 ; error = 482.9312


[(4, 494.6643359385029),
 (9, 494.39511733376276),
 (14, 493.87745470046127),
 (19, 493.3205205849925),
 (24, 492.76275902145636),
 (29, 492.3617586009616),
 (34, 491.81656126352783),
 (39, 491.29160256021265),
 (44, 490.87524747739826),
 (49, 490.4412320492535),
 (54, 490.033287972102),
 (59, 489.69771949061845),
 (64, 489.25809251567273),
 (69, 488.82950632988167),
 (74, 488.4153485827053),
 (79, 487.9820455111054),
 (84, 487.6732927488679),
 (89, 487.2768652110369),
 (94, 486.81999884037043),
 (99, 486.3581648312557),
 (104, 485.89866507032866),
 (109, 485.55335767650274),
 (114, 485.23974532261417),
 (119, 484.89725432095474),
 (124, 484.5846837677159),
 (129, 484.30035381862456),
 (134, 483.9524400515131),
 (139, 483.51471472972975),
 (144, 483.2590518923825),
 (149, 482.9312383234679),
 (154, 482.6023187896032),
 (159, 482.2175106922309),
 (164, 481.82787195879996),
 (169, 481.47803590952265),
 (174, 481.0627877100348),
 (179, 480.6945583071452),
 (184, 480.3819196588375),
 (189,

In [10]:
def predict(u, i):
    if u not in RA.user2index and i not in RA.item2index:
        return RA.all_avg
    if u not in RA.user2index:
        return sum([r for u,r in RA.item2user[i]]) / len(RA.item2user[i])
    if i not in RA.item2index:
        return sum([r for u,r in RA.user2item[u]]) / len(RA.user2item[u])
    
    user_index = RA.user2index[u]
    item_index = RA.item2index[i]
    return mfa.get_rating(user_index, item_index)

In [11]:
my_baselines.ratingBaseline(predict)